# Fix responses in the Continuous task

Something weird happened where the reaction times in the Continuous task are negative. For some reason, the feedback collection for the first trial never records a stop time.
This script corrects the responses by shifting participant answers to the previous trial.

It also discards the trials corresponding to the first two trials of each sweep.

For each participant (skipping over those that are listed in the `ignore_participants` list):

1. `fetch_initialization_data`: Initialization data is fetched for each participant.
2. `process_csv_files`: CSV files for each participant are processed and compiled into a single DataFrame.
3. `correct_data`: The data is corrected by addressing negative reaction times, removing catch trials, and discarding the first two trials of each sweep outside of the random condition. False alarm rates and detection ratios are computed and logged.
4. `recompute_audiograms`: external API call to recompute all four audiograms, using the initialization data and the corrected data.
5. `save_audiogram`: Each recomputed audiogram is saved in RAW_DATA/participant/Continuous with the suffix 'fixed'.

Finally, print stats on the effect of the corrections on the FAR and detection ratio.

In [1]:
from funcs.fix_continuous_responses import *

In [2]:
# Set path to raw data (csv files)
raw_data_path = get_path('raw_data')

participants = exclude_participants(os.listdir(raw_data_path))

overwrite_existing = True

In [3]:
API_access = get_API_access()

API_calls = {}
API_answer = {}
init_data = {}

# Keep track of whether detection and false alarm ratios improve
are_FARs_looking_better = []
are_DRs_looking_better = []

for participant in participants:  # ['ofgjwt']
    API_calls[participant] = {}
    API_answer[participant] = {}

    recomputed_files = filter_files_by_suffix(participant, 'Continuous', "*_fixed.pkl")

    if len(recomputed_files) < 4 or overwrite_existing:
        print("\n--------------------", participant, "--------------------")

        init_data[participant] = fetch_initialization_data(participant, raw_data_path)

        participant_data = fetch_paradigm_raw_data(participant, 'Continuous', correct_continuous=True, use_all_available_data=True)

        # Check false alarm rate previous to shifting
        old_FAR = check_false_alarm_rates(participant_data, shifted_responses=False)
        # Check false alarm rate after shifting
        new_FAR = check_false_alarm_rates(participant_data, shifted_responses=True)
        are_FARs_looking_better.append(new_FAR - old_FAR)

        # Remove catch trials
        participant_data = participant_data.loc[participant_data.isCatchTrial == 0]

        # Check detection ratios before and after corrections
        are_DRs_looking_better = check_detection_ratios(are_DRs_looking_better, participant_data)

        for pred, pred_group in participant_data.groupby('pred'):

            API_answer[participant][pred] = recompute_audiogram(participant, pred, pred_group, init_data[participant], API_calls, *API_access)

            save_audiogram(raw_data_path, participant, pred, API_answer[participant][pred])

        print("-------------------------------------------------")



-------------------- tlvjds --------------------
tlvjds_Continuous_2022-05-06_11h41.50.914_1.csv
New exp file

Fixing negative RTs for tlvjds: mean RT = -1.32, 167/194 RTs < 0
Filtering RTs < 1sec after the tone onset: 29 detections removed
-----
Removing first 2 trials of each sweep (=/= none): 286 -> 244 total trials
Original false alarm rate: 0.519
Shifted false alarm rate: 0.074
-----
Original detection ratio: 0.673
New detection ratio: 0.525
-----
Making request for tlvjds (both)
Saving recomputed audiogram: /Users/nadou/Projects/there_will_be_beeps/data/raw_data/tlvjds/Continuous/tlvjds_Continuous_both_fixed.pkl
Making request for tlvjds (frequency)
Saving recomputed audiogram: /Users/nadou/Projects/there_will_be_beeps/data/raw_data/tlvjds/Continuous/tlvjds_Continuous_frequency_fixed.pkl
Making request for tlvjds (none)
Saving recomputed audiogram: /Users/nadou/Projects/there_will_be_beeps/data/raw_data/tlvjds/Continuous/tlvjds_Continuous_none_fixed.pkl
Making request for tlvjds

/Users/nadou/Projects/there_will_be_beeps/scripts/utils.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc['feedback.rt'] = "[]"


KeyError: 'feedback.rt'

In [ ]:
# Check if we've "improved" FARs and detection rations with the corrections made
print("")
print(f"Improved (decreased) FARs for {np.sum(np.array(are_FARs_looking_better) < 0)} participants")
print(f"Mean improvement: {np.round(np.mean(are_FARs_looking_better) * 100)} %")
print("")
print(f"Decreased correct detection ratios for {np.sum(np.array(are_DRs_looking_better) < 0)} participants")
print(f"Mean improvement: {np.round(np.mean(are_DRs_looking_better) * 100)} %")
print("")